In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

hist_btc =  yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "BTC-USD",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max"

    )

hist_btc['daily_return'] = hist_btc.loc[:,['Close']].diff()/hist_btc.loc[:,['Close']].shift(1)
hist_btc['three_day_return'] = hist_btc.loc[:,['Close']].diff(3)/hist_btc.loc[:,['Close']].shift(3)
hist_btc['five_day_return'] = hist_btc.loc[:,['Close']].diff(5)/hist_btc.loc[:,['Close']].shift(5)
hist_btc['daily_up'] = hist_btc.daily_return.apply(lambda x: 1 if x>0 else 0)
hist_btc['three_day_up'] = hist_btc.three_day_return.apply(lambda x: 1 if x>0 else 0)
hist_btc['five_day_up'] = hist_btc.five_day_return.apply(lambda x: 1 if x>0 else 0)
#hist_eth['daily_return_lag'] = hist_eth.daily_return.shift(-1)
#hist_eth['five_day_return_lag'] = hist_eth.five_day_return.shift(-5)
print(hist_btc)
BTC_df_list = ['BTC_from_date_2016-01-01_to_date_2016-12-31.csv',
               'BTC_from_date_2017-01-01_to_date_2017-12-31.csv','from_date_2018-01-01_to_date_2018-12-31.csv',
               'from_date_2019-01-01_to_date_2019-12-31.csv','from_date_2020-01-01_to_date_2020-12-31.csv',
               'from_date_2021-01-01_to_date_2021-10-31.csv']

df_list = []
for file in BTC_df_list:
    temp = pd.read_csv(file,  index_col = 0)
    temp.replace([np.inf, -np.inf], np.nan, inplace=True)
    temp = temp.fillna(0)

    df_list.append(temp)
df = pd.concat(df_list, axis = 0, ignore_index = True)

start_date = datetime.date(2016, 1, 1)
end_date = datetime.date(2021, 10, 31)
delta = datetime.timedelta(days=1)

date_list = []

while start_date <= end_date:
    date_string = str(start_date)
    date_list.append(date_string)
    start_date += delta

df.index = date_list
df.set_index(pd.to_datetime(df.index), inplace=True)
#hist_eth.index = hist_eth.index.map(str)


merged_df = pd.merge(df,hist_btc, how='inner', left_index=True, right_index=True)
merged_df.to_csv('BTC_merged.csv')
print(merged_df)

[*********************100%***********************]  1 of 1 completed
                    Open          High  ...  three_day_up  five_day_up
Date                                    ...                           
2014-09-17    465.864014    468.174011  ...             0            0
2014-09-18    456.859985    456.859985  ...             0            0
2014-09-19    424.102997    427.834991  ...             0            0
2014-09-20    394.673004    423.295990  ...             0            0
2014-09-21    408.084991    412.425995  ...             0            0
...                  ...           ...  ...           ...          ...
2021-11-27  53736.429688  55329.257812  ...             0            0
2021-11-28  54813.023438  57393.843750  ...             0            0
2021-11-29  57291.906250  58872.878906  ...             1            1
2021-11-30  57830.113281  59113.402344  ...             1            0
2021-12-02  57025.238281  57150.335938  ...             0            1

[2629 r